In [12]:
from config import Mongouri
import pymongo

In [13]:
import pandas as pd

In [14]:
client = pymongo.MongoClient(Mongouri)

In [15]:
# !pip install dnspython

In [16]:
db = client.airportweather

In [17]:
def months (x):
    if x==1:
        return "January"
    elif x==2:
        return "February"
    elif x==3:
        return "March"
    elif x==4:
        return "April"
    elif x==5:
        return "May"
    elif x==6:
        return "June"
    elif x==7:
        return "July"
    elif x==8:
        return "August"
    elif x==9:
        return "September"
    elif x==10:
        return "October"
    elif x==11:
        return "November"
    elif x==12:
        return "December"
        

In [18]:
df = pd.read_csv("./Data/Combined_Cleaned.csv")
df["month"] = df["month"].map(months)
df_list = df.to_dict("record")
df.head()

,month,City,Air_code,State,Airport,arr_flights,arr_del15,carrier_ct,weather_ct,nas_ct,...,Precipitation,Rain,Snow,Storm,EventTotal,Avg_Severity,LocationLat,LocationLng,County,ZipCode
0,January,Aberdeen,ABR,SD,Aberdeen Regional,62,4,1.03,0.00,1.09,...,0,0,35,0,67,2.074627,45.4436,-98.4138,Brown,57401
1,January,Abilene,ABI,TX,Abilene Regional,28,9,4.94,0.00,2.28,...,0,15,1,1,30,2.500000,32.4113,-99.6819,Taylor,79602
2,January,Akron,CAK,OH,Akron-Canton Regional,504,96,23.83,3.90,31.53,...,0,64,45,0,134,1.447761,41.0375,-81.4642,Summit,44306
3,January,Albany,ABY,GA,Southwest Georgia Regional,82,20,9.22,0.00,2.46,...,1,68,0,0,80,1.812500,31.5365,-84.1957,Dougherty,31721
4,January,Albuquerque,ABQ,NM,Albuquerque International Sunport,1391,333,104.26,5.31,53.84,...,0,27,3,0,35,1.285714,35.0417,-106.6147,Bernalillo,87106


In [21]:
weather_df = df[["City","Airport", "State", "Cold", "Fog", "Hail", "Precipitation", "Rain", "Snow", "Storm", "EventTotal" , "LocationLat", "LocationLng"]].copy()
weather_df = weather_df.drop_duplicates(subset = ["Airport"])
gb = weather_df.groupby(["City","Airport", "State","LocationLat","LocationLng"]).sum()
weather_list = pd.DataFrame(gb).reset_index().rename(columns={"EventTotal": "Total Weather" , "LocationLat": "Lat" , "LocationLng": "Lng" }).to_dict("record")
weather_list

[{'City': 'Aberdeen',
  'Airport': 'Aberdeen Regional',
  'State': 'SD',
  'Lat': 45.4436,
  'Lng': -98.4138,
  'Cold': 8,
  'Fog': 24,
  'Hail': 0,
  'Precipitation': 0,
  'Rain': 0,
  'Snow': 35,
  'Storm': 0,
  'Total Weather': 67},
 {'City': 'Abilene',
  'Airport': 'Abilene Regional',
  'State': 'TX',
  'Lat': 32.4113,
  'Lng': -99.6819,
  'Cold': 4,
  'Fog': 9,
  'Hail': 0,
  'Precipitation': 0,
  'Rain': 15,
  'Snow': 1,
  'Storm': 1,
  'Total Weather': 30},
 {'City': 'Akron',
  'Airport': 'Akron-Canton Regional',
  'State': 'OH',
  'Lat': 41.0375,
  'Lng': -81.4642,
  'Cold': 0,
  'Fog': 25,
  'Hail': 0,
  'Precipitation': 0,
  'Rain': 64,
  'Snow': 45,
  'Storm': 0,
  'Total Weather': 134},
 {'City': 'Albany',
  'Airport': 'Southwest Georgia Regional',
  'State': 'GA',
  'Lat': 31.5365,
  'Lng': -84.1957,
  'Cold': 0,
  'Fog': 11,
  'Hail': 0,
  'Precipitation': 1,
  'Rain': 68,
  'Snow': 0,
  'Storm': 0,
  'Total Weather': 80},
 {'City': 'Albuquerque',
  'Airport': 'Albuquerqu

In [22]:
# gb = weather_list.groupby(["Airport" , "City"]).count()["Lat"]
# df = pd.DataFrame(gb).reset_index()
# df.loc[df["Lat"]>1]

In [10]:
for row in df_list:
    db.WeatherDelayInfo.insert_one(row)
    

In [23]:
for row in weather_list:
    db.WeatherTop10.insert_one(row)